In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


# 2. Scrap data from Wikipedia page into a DataFrame

In [9]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [10]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [11]:
# create a list to store neighborhood data
neighborhoodList = []

In [12]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [13]:
# create a new DataFrame from the list
mmb_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mmb_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


# 3. Get the geographical coordinates

In [14]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [18]:
# call the function to get the coordinates, store in a new list using list comprehension
!conda install -c conda-forge geocoder --yes 
import geocoder
coords = [ get_latlng(neighborhood) for neighborhood in mmb_df["Neighborhood"].tolist() ]

Solving environment: ...working... done

# All requested packages already installed.



In [19]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [20]:
# merge the coordinates into the original dataframe
mmb_df['Latitude'] = df_coords['Latitude']
mmb_df['Longitude'] = df_coords['Longitude']

In [22]:
# check the neighborhoods and the coordinates
print(mmb_df.shape)
mmb_df

(41, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.119770,72.846290
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.053995,72.899675
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


# 4. Create a map of Mumbai with neighborhoods superimposed on top

In [26]:
# Use geopy library to get the latitude and longitude values of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 19.0759899, 72.8773928.


In [35]:
# create map of Mumbai using latitude and longitude values
map_Mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng,  neighborhood in zip(mmb_df['Latitude'], mmb_df['Longitude'],  mmb_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Mumbai)  
    
map_Mumbai

# 5. Use the Foursquare API to explore the neighborhoods

In [36]:
# Add Foursquare credentials to run remainder of notebook
CLIENT_ID = 'UFDVP4GDWUZ4CH3H32SWPLOGJYBA2FY3QNE0OV550XCXWOVR' # your Foursquare ID
CLIENT_SECRET = 'EFM0VOIWD0CPPBE0Q0JWKPDTONEYLGCK14V4YWENJBEGJQ1C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Get the top 100 venues in a radius of 2500 meters

In [37]:
radius = 2500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mmb_df['Latitude'], mmb_df['Longitude'], mmb_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [38]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2870, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.11977,72.84629,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.11977,72.84629,Narayan Sandwich,19.121398,72.850270,Sandwich Place
2,Andheri,19.11977,72.84629,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
3,Andheri,19.11977,72.84629,Naturals,19.111204,72.837255,Ice Cream Shop
4,Andheri,19.11977,72.84629,Shawarma Factory,19.124591,72.840398,Falafel Restaurant


In [39]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,19,19,19,19,19,19
Baiganwadi,4,4,4,4,4,4
Bandra,100,100,100,100,100,100
Bhandup,35,35,35,35,35,35
Borivali,100,100,100,100,100,100
Charkop,62,62,62,62,62,62
Chembur,92,92,92,92,92,92
Dahisar,76,76,76,76,76,76


In [40]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

There are 186 uniques categories.


True

# 6. Analyze Each Neighborhood

In [41]:
# one hot encoding
mmb_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mmb_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mmb_onehot.columns[-1]] + list(mmb_onehot.columns[:-1])
mmb_onehot = mmb_onehot[fixed_columns]

print(mmb_onehot.shape)
mmb_onehot.head()

(2870, 187)


,Neighborhoods,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [43]:
mmb_grouped = mmb_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mmb_grouped.shape)
mmb_grouped

(40, 187)


,Neighborhoods,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Andheri,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.010000,0.00,0.020000,0.040000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.010000,0.010000,0.00,0.00,0.000000,0.000000,0.000000,0.050000,0.00,0.00,0.050000,0.000000,0.020000,0.010000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.010000,0.00,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.020000,0.000000,0.060000,0.000000,0.180000,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.030000,0.000000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.010000,0.00,0.00000,0.00,0.050000,0.00,0.000000,0.000000,0.000000,0.030000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.030000,0.000000,0.050000,0.000000,0.000000,0.000000,0.

In [44]:
len(mmb_grouped[mmb_grouped["Shopping Mall"] > 0])

19

# Create a new DataFrame for Shopping Mall data only

In [45]:
mmb_mall = mmb_grouped[["Neighborhoods","Shopping Mall"]]
mmb_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.028571


# 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [46]:
# set number of clusters
kclusters = 3

mmb_clustering = mmb_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mmb_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 2, 0, 1, 1, 1, 2])

In [47]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mmb_merged = mmb_mall.copy()

# add clustering labels
mmb_merged["Cluster Labels"] = kmeans.labels_

In [48]:
mmb_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mmb_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.000000,1
1,Anushakti Nagar,0.000000,1
2,Baiganwadi,0.000000,1
3,Bandra,0.000000,1
4,Bhandup,0.028571,2


In [49]:
# merge mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
mmb_merged = mmb_merged.join(mmb_df.set_index("Neighborhood"), on="Neighborhood")

print(mmb_merged.shape)
mmb_merged.head() # check the last columns!

(40, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,1,19.11977,72.84629
1,Anushakti Nagar,0.000000,1,19.04283,72.92734
2,Baiganwadi,0.000000,1,19.06294,72.92663
3,Bandra,0.000000,1,19.05437,72.84017
4,Bhandup,0.028571,2,19.14556,72.94856


In [50]:
# sort the results by Cluster Labels
print(mmb_merged.shape)
mmb_merged.sort_values(["Cluster Labels"], inplace=True)
mmb_merged

(40, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
17,Kandivali,0.010000,0,19.204160,72.851690
37,Wadala,0.010638,0,19.017200,72.858160
35,Vashi,0.011111,0,19.080060,72.998610
21,Mahavir Nagar (Kandivali),0.010000,0,19.210940,72.841370
5,Borivali,0.010000,0,19.229360,72.857510
31,"Sonapur, Bhandup",0.014493,0,19.163940,72.935440
20,Kurla,0.010000,0,19.064980,72.880690
27,Pestom sagar,0.010000,0,19.070640,72.902170
22,Mankhurd,0.000000,1,19.048530,72.932220
23,"Matharpacady, Mumbai",0.000000,1,18.940170,72.834830


# visualize the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mmb_merged['Latitude'], mmb_merged['Longitude'], mmb_merged['Neighborhood'], mmb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 8. Examine Clusters

Cluster 0

In [52]:
mmb_merged.loc[mmb_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
17,Kandivali,0.010000,0,19.20416,72.85169
37,Wadala,0.010638,0,19.01720,72.85816
35,Vashi,0.011111,0,19.08006,72.99861
21,Mahavir Nagar (Kandivali),0.010000,0,19.21094,72.84137
5,Borivali,0.010000,0,19.22936,72.85751
31,"Sonapur, Bhandup",0.014493,0,19.16394,72.93544
20,Kurla,0.010000,0,19.06498,72.88069
27,Pestom sagar,0.010000,0,19.07064,72.90217


Cluster 1

In [53]:
mmb_merged.loc[mmb_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
22,Mankhurd,0.0,1,19.048530,72.932220
23,"Matharpacady, Mumbai",0.0,1,18.940170,72.834830
24,Mira Road,0.0,1,19.265705,72.870693
0,Andheri,0.0,1,19.119770,72.846290
28,Seven Bungalows,0.0,1,19.128560,72.821180
30,"Sion, Mumbai",0.0,1,19.043410,72.863320
33,Tilak Nagar (Mumbai),0.0,1,18.996160,72.852790
34,Uttan,0.0,1,19.260040,72.793412
26,Mumbra,0.0,1,19.188413,73.022011
29,Shil Phata,0.0,1,19.146580,73.040050


Cluster 2

In [54]:
mmb_merged.loc[mmb_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
36,Vikhroli,0.023256,2,19.111090,72.927810
4,Bhandup,0.028571,2,19.145560,72.948560
32,Thakur village,0.046154,2,19.210200,72.875410
38,Western Suburbs (Mumbai),0.023256,2,19.197010,72.827680
9,Devipada,0.020000,2,19.224690,72.866050
11,Eastern Suburbs (Mumbai),0.024691,2,19.004272,72.855790
25,Mulund,0.020000,2,19.171830,72.955650
12,Ghatkopar,0.020000,2,19.085357,72.908623
13,Goregaon,0.020000,2,19.164550,72.849460
18,Kanjurmarg,0.028571,2,19.131380,72.935680


Observations:

Most of the shopping malls are concentrated in the central area of Mumbai city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.